In [19]:
from torch.utils.data import DataLoader
from tqdm import tqdm

from context import kolihack
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import pandas as pd

from kolihack.io import load_content_file

In [21]:
df = load_content_file()

Successfully loaded pandas data frame from C:\Users\jenni\workspace\kolihack\data/content.csv


In [22]:
df.describe()

,id,title,description,kind,text,language,copyright_holder,license
count,154047,154038,89456,154047,74035,154047,71821,74035
unique,154047,130937,76305,5,70687,27,148,7
top,c_00002381196d,Video,v0.1,video,Unsupported browser\n\nThe HTML5 content is no...,en,Khan Academy,CC BY-NC-SA
freq,1,504,903,61487,234,65939,17034,52088


In [23]:
df

,id,title,description,kind,text,language,copyright_holder,license
0,c_00002381196d,"Sumar números de varios dígitos: 48,029+233,930","Suma 48,029+233,930 mediante el algoritmo está...",video,NaN,es,NaN,NaN
1,c_000087304a9e,Trovare i fattori di un numero,Sal trova i fattori di 120.\n\n,video,NaN,it,NaN,NaN
2,c_0000ad142ddb,Sumar curvas de demanda,Cómo añadir curvas de demanda\n\n,video,NaN,es,NaN,NaN
3,c_0000c03adc8d,Nado de aproximação,Neste vídeo você vai aprender o nado de aproxi...,document,\nNado de aproximação\nSaber nadar nas ondas ...,pt,Sikana Education,CC BY-NC-ND
4,c_00016694ea2a,geometry-m3-topic-a-overview.pdf,geometry-m3-topic-a-overview.pdf,document,Estándares Comunes del Estado de Nueva York\n\...,es,Engage NY,CC BY-NC-SA
...,...,...,...,...,...,...,...,...
154042,c_fffcbdd4de8b,2. 12: Diffusion,NaN,html5,What will eventually happen to these dyes?\n\n...,en,CSU and Merlot,CC BY-NC-SA
154043,c_fffe15a2d069,Sommare facendo gruppi da 10,Sal somma 5+68 spezzando il 5 in un 2 e un 3.\n\n,video,NaN,it,NaN,NaN
154044,c_fffed7b0d13a,Introdução à subtração,Sal fala sobre o que significa subtrair. Os ex...,video,NaN,pt,NaN,NaN
154045,c_ffff04ba7ac7,SA of a Cone,NaN,video,NaN,en,NaN,NaN


In [24]:
t1 = "Concentration within the economy is increasing"
t2 = "Fortune 500 companies represent 2/3 of US GDP"
t3 = "The US economy is more concentrated within major firms"
t4 = "Adoption of machine learning is increasing at many companies"
t5 = "Firms are building more AI products"

In [25]:
import numpy as np
from transformers import pipeline
from sklearn.metrics.pairwise import cosine_similarity

In [26]:
def normalized_mean_pooling(token_vectors):
    sentences_vectors = [np.mean(tokens, axis=0) for tokens in token_vectors]
    normalized_embeddings = [vector / np.linalg.norm(vector) \
                    for vector in sentences_vectors]
    return normalized_embeddings


In [27]:

def encoder(texts, modelname):
    '''
    Use huggingface pipeline class to get vector embeddings for each token,
    then take the mean across tokens to get one vector embbedding per text
    '''
    pipe = pipeline("feature-extraction",
                   model=modelname,
                   tokenizer=modelname)

    embeddings = []
    loader = DataLoader(texts, batch_size=32, shuffle=False)
    for inputs in tqdm(loader):
        vectors = pipe(inputs)
        vectors = [np.vstack(item) for item in vectors]
        embs = normalized_mean_pooling(vectors)
        embeddings.extend(embs)
    return embeddings


In [48]:
description_ = df['description']
non_nan_descriptions = description_.dropna(ignore_index=True)

pydev debugger: Unable to find real location for: C:\Users\jenni\AppData\Local\Temp\ipykernel_11504\155152298.py


In [49]:
non_nan_descriptions = non_nan_descriptions[0:10]
input_embedding = encoder(non_nan_descriptions, "bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 1/1 [00:01<00:00,  1.07s/it]


In [51]:
df_i = pd.DataFrame({'description': input_embedding})
df_i.to_csv('input_embeddings.csv')

pydev debugger: Unable to find real location for: C:\Users\jenni\AppData\Local\Temp\ipykernel_11504\1515378759.py
pydev debugger: Unable to find real location for: C:\Users\jenni\AppData\Local\Temp\ipykernel_11504\522604472.py


In [53]:
# question = "¿qué son las curvas de demanda?"
question = "Cómo añadir curvas de demanda\n\n"

question_embedding = encoder([question], "bert-base-uncased")

similarities_bert = cosine_similarity(input_embedding, question_embedding)

print(similarities_bert)
t = "*4"

pydev debugger: Unable to find real location for: C:\Users\jenni\AppData\Local\Temp\ipykernel_11504\4031721847.py
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 1/1 [00:00<00:00,  9.14i

[[0.78164127]
 [0.53997634]
 [1.        ]
 [0.57507046]
 [0.35939738]
 [0.32642676]
 [0.82206537]
 [0.38561224]
 [0.246483  ]
 [0.28875162]]
